# Faster R-CNN on Custom Data

## Step 1: Install OpenCV Packages for Image Processing: Uninstall, Update, and Install Dependencies
- Uninstall the **opencv-python** package using pip
- Install the **glib** package with a specific version (2.51.0) using conda
- Install the **opencv-python** package using pip
- Install the **opencv-python-headless** package using pip

**Note:** Install these packages only when using a local machine, not the Simplilearn lab


In [ ]:
#!pip uninstall -y opencv-python
#!conda install glib=2.51.0 -y
#!pip install opencv-python
#!pip install opencv-python-headless

## Step 2: Import Necessary Libraries

- Import the OpenCV library for image processing
- Import the NumPy library for numerical operations


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Step 3: Download YOLOv3 Pretrained Weights for Object Detection
- Download YOLOv3 pretrained weights


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

## Step 4: Load and Preprocess an Image for Object Detection

- Read the image from the file
- Get the width and height of the image
- Define the scale factor for image preprocessing


In [ ]:
image = cv2.imread("test.jpg")
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392


## Step 5: Load Class Names and Generate Random Colors for Object Detection Labels
- Load the class names from the **coco.names** file
- Generate random colors for each class



In [ ]:
classes = None
with open("coco.names", 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

## Step 6: Load Pretrained YOLOv3 Model for Object Detection

In [ ]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

## Step 7: Generate Blob for YOLOv3 Object Detection
- Preprocess the image to obtain a blob
- Set the input blob for the YOLOv3 network

In [ ]:
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)


In [ ]:
net.setInput(blob)

## Step 8: Get Output Layers from YOLOv3 Network and Extract Names of Output Layers
- Retrieve the names of all the layers in the given neural network model
- Retrieve the indices of the layers which are not connected to subsequent (output) layers
- Return the list of output layer names



In [ ]:
def get_output_layers(net):

    layer_names = net.getLayerNames()

    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

## Step 9: Draw Bounding Boxes and Labels on Images

- Get the class label corresponding to the class_id
- Get the color for drawing the bounding box
- Draw the bounding box rectangle on the image
- Put the class label text on the image



In [ ]:
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

## Step 10: Process YOLOv3 Output: Filtering Detections and Extract Bounding Box Information
- Perform a forward pass through the neural network to get predictions
- Initialize lists for detected class IDs, confidence scores, and bounding boxes
- For each detection, retrieve scores of all classes
- Identify the class with the highest score
- If confidence is greater than 0.5, compute bounding box details
- Append class ID, confidence, and bounding box to their respective lists


In [ ]:
outs = net.forward(get_output_layers(net))

class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

## Step 11: Apply Non-Maximum Suppression (NMS) and Draw Bounding Boxes on Detected Objects
- Go through the remaining detections after non-maximum suppression (NMS) and draw bounding boxes
- Get the box coordinates
- Draw the bounding box on the image


In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

for i in indices:
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]

    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

## Step 12: Display the Output Image
- Display the image with bounding boxes using matplotlib
- Save the image with bounding boxes to a file


In [ ]:
plt.imshow(image)
cv2.imwrite("test.jpg", image)

**Observation:**
- This code efficiently processes the image, detects objects, draws bounding boxes around the identified objects, and saves the results.